### KITTI_Raw to poses (runtime: ~ 1min)

In [6]:
import numpy as np
import argparse
import math
import os

#### Set paths
<div class="alert alert-block alert-warning">
<b>ToDo:</b> 
<ol>
    <li> Set "datasets_root" to the root of your <b>dataset directory</b>
    <li> Set "vanishing_data_root" to your <b>working directory</b>
</ol>
</div>

In [7]:
datasets_root = '/disk/ml/datasets'
vanishing_data_root = f'/disk/vanishing_data/ju878'

In [8]:
kitti_raw_root = os.path.join(datasets_root, 'KITTI_Raw')

prepared_poses = os.path.join(vanishing_data_root, 'prepared_poses')
poses_folder = os.path.join(prepared_poses, 'kitti/poses')

In [9]:
OXTS_LINE_LEN = 30

In [10]:
if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)

#### Functions to create poses

Load timestamps

In [11]:
def load_timestamps(ts_dir):
    ts_file = os.path.join(ts_dir, 'timestamps.txt')
    return np.loadtxt(ts_file, delimiter=",", converters={0: lambda v: np.int64(np.datetime64(v)) / 1e+9 })

Load oxts

In [12]:
def load_oxts_data(base_dir):
    oxts_dir = os.path.join(base_dir, 'oxts')
    ts = load_timestamps(oxts_dir)
    ts_len = len(ts)
    oxts_data = np.zeros((ts_len, OXTS_LINE_LEN))
    for i in range(ts_len):
        data_dir = os.path.join(oxts_dir, 'data')
        t_filename = str(i).zfill(10) + ".txt"
        oxts_data[i] = np.loadtxt(os.path.join(data_dir, t_filename))
    return oxts_data

Helper functions

In [13]:
def lat_lon_to_mercator(lat, lon, scale):
    ER = 6378137
    mx = scale * lon * math.pi * ER / 180
    my = scale * ER * math.log(math.tan((90 + lat) * math.pi / 360))
    return mx, my


def lat_to_scale(lat):
    return math.cos(lat * math.pi / 180.0)

Translate oxts to poses

In [14]:
def gps_imu_to_pose(gps_imu, scale):
    t = np.zeros(3)
    t[0], t[1] = lat_lon_to_mercator(gps_imu[0], gps_imu[1], scale)
    t[2] = gps_imu[2]  # altitude
    rx = gps_imu[3]  # roll
    ry = gps_imu[4]  # pitch
    rz = gps_imu[5]  # heading
    Rx = np.array([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]])
    Ry = np.array([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]])
    Rz = np.array([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def oxts_to_pose(oxts_info):
    # Compute scale from first lat value
    scale = lat_to_scale(oxts_info[0, 0])
    Tr_0_inv = None
    poses = np.zeros((len(oxts_info), 12))
    for i, line in enumerate(oxts_info):
        T = gps_imu_to_pose(line, scale)
        # Normalize translation and rotation (start at 0/0/0)
        if Tr_0_inv is None:
            Tr_0_inv = np.linalg.inv(T)

        pose = Tr_0_inv.dot(T)
        poses[i] = pose[:3, :].reshape(12)
    return poses

#### Translate KITTI_oxts to KITTI_poses

In [19]:
dates = os.listdir(kitti_raw_root)
for date in dates:
    date_root = os.path.join(kitti_raw_root, date)
    sequences = os.listdir(date_root)
    for seq in sequences:
        seq_root = os.path.join(date_root, seq)
        if os.path.isdir(seq_root):
            oxts_data = load_oxts_data(seq_root)
            poses = oxts_to_pose(oxts_data)
            poses_path = os.path.join(poses_folder, date, seq)
            if not os.path.exists(poses_path):
                os.makedirs(poses_path)
            np.savetxt(os.path.join(poses_path, 'poses.txt'), poses)